In [71]:
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from scipy.stats import kurtosis
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

# measurement = np.load('../../dataset/meas_symm_1.npz', allow_pickle=False)
# measurement = np.load('../../dataset/meas_symm_2.npz', allow_pickle=False)
# measurement = np.load('../../dataset/meas_symm_3.npz', allow_pickle=False)
# measurement = np.load('../../dataset/meas_symm_4.npz', allow_pickle=False)
# measurement = np.load('../../dataset/meas_symm_5.npz', allow_pickle=False)
# measurement = np.load('../../dataset/meas_symm_nomove_1.npz', allow_pickle=False)
# measurement = np.load('../../dataset/meas_symm_varspeed_1.npz', allow_pickle=False)

# measurement = np.load('../../dataset/meas_asymm_1.npz', allow_pickle=False)
# measurement = np.load('../../dataset/meas_asymm_2.npz', allow_pickle=False)
# measurement = np.load('../../dataset/meas_asymm_nomove_1.npz', allow_pickle=False)
measurement = np.load('../../dataset/meas_asymm_reflector_1.npz', allow_pickle=False)

header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:8000]
trainCIR, testCIR = train_test_split(data_cir, test_size=0.2, random_state=42)
print(trainCIR.shape, testCIR.shape)

(6400, 15, 251, 2) (1600, 15, 251, 2)


In [72]:
def get64Samples(cirs):

    real = cirs[:, :, 0]
    imag = cirs[:, :, 1]
    
    # Number of signalsd
    num_signals = real.shape[0]  # 3 in this case
    # print(f'num_signals: {num_signals}')
    # Initialize lists to store the focused samples
    imp_real_parts = []
    imp_imag_parts = []
    img_mag_parts = []
    
    for i in range(num_signals):
        # Calculate the magnitude
        magnitude = np.abs(real[i] + 1j * imag[i])
        
        # find the peak index
        peak_index = np.argmax(magnitude)
        
        # Calculate the start and end indices for the focused part
        start_index = max(0, peak_index - 32)
        end_index = min(magnitude.shape[0], peak_index + 32)
        
        # Extract the part of the signal around the peak
        real_part_focus = real[i, start_index:end_index]
        imag_part_focus = imag[i, start_index:end_index]
        mag_part_focus = magnitude[start_index:end_index]
        
        imp_real_parts.append(real_part_focus)
        imp_imag_parts.append(imag_part_focus)
        img_mag_parts.append(mag_part_focus)
        

    # Convert lists back to arrays for further processing if needed
    imp_real_parts = np.array(imp_real_parts)
    imp_imag_parts = np.array(imp_imag_parts)
    img_mag_parts = np.array(img_mag_parts)

    return imp_real_parts, imp_imag_parts, img_mag_parts


def calculate_crest_factor(signal):
    peak = np.max(np.abs(signal), axis=1)
    rms = np.linalg.norm(signal, axis=1) / np.sqrt(signal.shape[1])
    rms = np.clip(rms, a_min=np.finfo(float).eps, a_max=None)
    return peak / rms

def getRealImaginaryMagnitudeParts(cirs):
    real = cirs[:, :, 0]
    imag = cirs[:, :, 1]
    mag = np.abs(real + 1j * imag)
    return real, imag, mag

In [73]:
# Define channels
alice_channel = 3  # Channel 3 is ALICE (legitimate)
eve_channel = 6  # Channel 6 is EVE (illegitimate)

# Data extraction
alice_CIRs = trainCIR[:, alice_channel, :, :]  # Shape: (9797, 251, 2)
eve_CIRs = trainCIR[:, eve_channel, :, :]  # Shape: (9797, 251, 2)

# Alice features:
# Real, imaginary, magnitude
alice_real, alice_imag, alice_mag = getRealImaginaryMagnitudeParts(alice_CIRs)
# Kurtosis
alice_kurtosis_real = kurtosis(alice_real, axis=1).reshape(-1, 1)  # Kurtosis of each signal in the real part
alice_kurtosis_imag = kurtosis(alice_imag, axis=1).reshape(-1, 1)  # Kurtosis of each signal in the imaginary part
# Crest factor
alice_crest_factor_real = calculate_crest_factor(alice_real).reshape(-1, 1)
alice_crest_factor_imag = calculate_crest_factor(alice_imag).reshape(-1, 1)
alice_features = np.hstack((alice_real, alice_imag, alice_mag, alice_kurtosis_real, alice_kurtosis_imag, alice_crest_factor_real, alice_crest_factor_imag))


# Eve features: 
# Real, imaginary, magnitude
eve_real, eve_imag, eve_mag = getRealImaginaryMagnitudeParts(eve_CIRs)
# kurtosis
eve_kurtosis_real = kurtosis(eve_real, axis=1).reshape(-1, 1)  # Kurtosis of each signal in the real part
eve_kurtosis_imag = kurtosis(eve_imag, axis=1).reshape(-1, 1)  # Kurtosis of each signal in the imaginary part
# Crest factor
eve_crest_factor_real = calculate_crest_factor(eve_real).reshape(-1, 1)
eve_crest_factor_imag = calculate_crest_factor(eve_imag).reshape(-1, 1)
# total features
eve_features = np.hstack((eve_real, eve_imag, eve_mag, eve_kurtosis_real, eve_kurtosis_imag, eve_crest_factor_real, eve_crest_factor_imag))


# Create labels for Alice and Eve.
alice_labels = np.zeros(alice_features.shape[0])  # Label '0' for Alice. (9797,)
eve_labels = np.ones(alice_features.shape[0])     # Label '1' for Eve. (9797,)

# Combine data and labels.
atoms = np.vstack((alice_features, eve_features))  # (19594, 192)
true_labels = np.hstack((alice_labels, eve_labels))  # (19594,)


# Combine data and labels for training
train_atoms = np.vstack((alice_features, eve_features))
train_labels = np.hstack((alice_labels, eve_labels))


# Apply PCA for Dimensionality Reduction
scaler = StandardScaler()
train_atoms_normalized = scaler.fit_transform(train_atoms)
print(f'atoms normalized : {train_atoms_normalized.shape}')

pca = PCA(n_components=250)
train_atoms_pca = pca.fit_transform(train_atoms_normalized)
print(f'atoms normalized pca : {train_atoms_pca.shape}')

D = train_atoms_pca.T

atoms normalized : (12800, 757)
atoms normalized pca : (12800, 250)


In [74]:
# Step 3: Extract Features for Test Data
alice_test_CIRs = testCIR[:, alice_channel, :, :]
eve_test_CIRs = testCIR[:, eve_channel, :, :]

# Alice test features:
# Real, imaginary, magnitude
alice_test_real, alice_test_imag, alice_test_mag = getRealImaginaryMagnitudeParts(alice_test_CIRs)
# Kurtosis
alice_test_kurtosis_real = kurtosis(alice_test_real, axis=1).reshape(-1, 1)  # Kurtosis of each signal in the real part
alice_test_kurtosis_imag = kurtosis(alice_test_imag, axis=1).reshape(-1, 1)  # Kurtosis of each signal in the imaginary part
# Crest factor
alice_test_crest_factor_real = calculate_crest_factor(alice_test_real).reshape(-1, 1)
alice_test_crest_factor_imag = calculate_crest_factor(alice_test_imag).reshape(-1, 1)
# total features
alice_test_features = np.hstack((alice_test_real, alice_test_imag, alice_test_mag, alice_test_kurtosis_real, alice_test_kurtosis_imag, alice_test_crest_factor_real, alice_test_crest_factor_imag))



# Eve test features:
# Real, imaginary, magnitude
eve_test_real, eve_test_imag, eve_test_mag = getRealImaginaryMagnitudeParts(eve_test_CIRs)
# Kurtosis
eve_test_kurtosis_real = kurtosis(eve_test_real, axis=1).reshape(-1, 1)  # Kurtosis of each signal in the real part
eve_test_kurtosis_imag = kurtosis(eve_test_imag, axis=1).reshape(-1, 1)  # Kurtosis of each signal in the imaginary part
# Crest factor
eve_test_crest_factor_real = calculate_crest_factor(eve_test_real).reshape(-1, 1)
eve_test_crest_factor_imag = calculate_crest_factor(eve_test_imag).reshape(-1, 1)
# total features
eve_test_features = np.hstack((eve_test_real, eve_test_imag, eve_test_mag, eve_test_kurtosis_real, eve_test_kurtosis_imag, eve_test_crest_factor_real, eve_test_crest_factor_imag))


# Create labels for Alice and Eve for testing
alice_test_labels = np.zeros(alice_test_features.shape[0])  # Label '0' for Alice.
eve_test_labels = np.ones(eve_test_features.shape[0])       # Label '1' for Eve.

# Combine data and labels for testing
test_atoms = np.vstack((alice_test_features, eve_test_features))
test_labels = np.hstack((alice_test_labels, eve_test_labels))

test_atoms_normalized = scaler.transform(test_atoms)
test_atoms_pca = pca.transform(test_atoms_normalized)

In [75]:
# Step 5: Sparse Coding Function
def find_sparse_coefficients(tSample, D, n_nonzero_coefs=20):
    # print(f'tSample: {tSample.shape}')
    omp = OrthogonalMatchingPursuit(n_nonzero_coefs=n_nonzero_coefs)
    omp.fit(D, tSample)
    return omp.coef_

# Step 6: Function to Calculate Residuals for Each Class
def calculate_residual(tSample, D, coefficients, class_indices):
    coef_class = np.zeros_like(coefficients)
    coef_class[class_indices] = coefficients[class_indices]  # Keep onltSample coefficients for the specified class
    reconstructed_signal = D @ coef_class
    residual = np.linalg.norm(tSample - reconstructed_signal)
    return residual

# Step 7: Classification Function
def classify_signal(tSample, D, trainLabel):
    
    coefficients = find_sparse_coefficients(tSample, D)

    residuals = []

    # Calculate residual for each class
    unique_classes = np.unique(trainLabel) # 0 and 1
    for class_label in unique_classes:
        class_indices = np.where(trainLabel == class_label)[0]  # Indices of columns in D belonging to this class
        residual = calculate_residual(tSample, D, coefficients, class_indices)
        residuals.append(residual)

    # Predict the class with the smallest residual
    predicted_class = unique_classes[np.argmin(residuals)]
    return predicted_class



In [76]:
# Step 8: Classifying Test Data and Evaluating the Model
predictions = []

for testSample in test_atoms_pca:
    predicted_class = classify_signal(testSample, D, train_labels)
    predictions.append(predicted_class)


predictions = np.array(predictions)
# print(predictions.shape)

accuracy = np.mean(predictions == test_labels)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

Classification Accuracy: 60.34%


In [77]:

# Calculate confusion matrix
print(f"\nTotal testing channel: {test_labels.shape}")

tn, fp, fn, tp = confusion_matrix(test_labels, predictions, labels=[0, 1]).ravel()
print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# Missed Detection Rate (MDR)
MDR = fp / (fp + tn)

# False Alarm Rate (FAR)
FAR = fn / (fn + tp)

# Gamma calculation
gamma = (tp + fn) / (tn + fp)

# Authentication Rate (AR)
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp))

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")



Total testing channel: (3200,)
tp: 1570
tn: 361
fp: 1239
fn: 30
MDR: 0.774375
FAR: 0.01875
AR: 0.6034375
